In [3]:
import pandas as pd
import os

# Define base data directory
RAW_DATA_DIR = os.path.abspath("../../data/raw")

# Load datasets
nd_transactions = pd.read_csv(os.path.join(RAW_DATA_DIR, "nonderiv_transactions.csv"))
print("Non-derivative transactions shape: ", nd_transactions.shape)
assert nd_transactions.shape == (165918, 13)

Non-derivative transactions shape:  (165918, 13)


In [ ]:
import pandas as pd
import os

# Define base data directory
RAW_DATA_DIR = os.path.abspath("../../data/raw")

# Load datasets
nd_transactions = pd.read_csv(os.path.join(RAW_DATA_DIR, "nonderiv_transactions.csv"))
print("Non-derivative transactions shape: ", nd_transactions.shape)
assert nd_transactions.shape == (165918, 13)